In [46]:
import numpy as np
import pandas as pd
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
import string
from sklearn.model_selection import KFold
from decimal import *
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import svm

## Cleaning the Data

In [10]:
# transactions.csv and transactions2.csv are data gathered from the same experiment. However, transactions2.csv is the result of the one that has more randomness to the generated transactions.

df = pd.read_csv('data/transactions2.csv')



In [6]:
# The following functions may be used in case we need to do a data transformation into categorical data.

def gas_bining(org_df):
    df = org_df.copy()
    
    df['gas_used'] = pd.cut(df['gas_used'], bins=[i * 100000 for i in range(0, 20)], labels=list(string.ascii_uppercase[:19]))
    
    return df


def transform_balance_delta(org_df):
    df = org_df.copy()


    for i in range(0, df.shape[0]):

        if int(df.at[i, 'victim_balance_delta']) == 0:
            df.at[i, 'victim_balance_delta'] = 'zero'

        elif int(df.at[i, 'victim_balance_delta']) > 0:
            df.at[i, 'victim_balance_delta'] = 'positive'

        elif int(df.at[i, 'victim_balance_delta']) < 0:
            df.at[i, 'victim_balance_delta'] = 'negative'


        if int(df.at[i, 'attacker_balance_delta']) == 0:
            df.at[i, 'attacker_balance_delta'] = 'zero'

        elif int(df.at[i, 'attacker_balance_delta']) > 0:
            df.at[i, 'attacker_balance_delta'] = 'positive'

        elif int(df.at[i, 'attacker_balance_delta']) < 0:
            df.at[i, 'attacker_balance_delta'] = 'negative'

        
    return df


def call_stack_depth_bining(org_df):
    df = org_df.copy()

    df['call_stack_depth'] = pd.cut(df['call_stack_depth'], bins=[i for i in range(0, 20)], labels=list(string.ascii_uppercase[:19]))
    
    return df



new_df = gas_bining(df)
new_df = transform_balance_delta(new_df)
new_df = call_stack_depth_bining(new_df)




In [64]:
# Helper functions

def get_ys_from_prediction(prediction_df):
    df = prediction_df.copy()
    ys = list()

    col_list = df.columns
    for index, row in df.iterrows():
        max_prob = row.max()
        for col in col_list:
            if row[col] == max_prob:
                predicted = col
                break
        ys.append(predicted)
    
    return ys

In [7]:
# An slightly manipulated version of the Random Forest written for course assignment


class RandomForest:

    def __init__(self):
        self.column_filter = None
        self.imputation = None 
        self.one_hot = None 
        self.labels = None 
        self.model = None
        

        # HELPER
        self.no_trees = None

        # DEBUG
        self.tree_predictions_list = None

    
    def fit(self, df, no_trees=100):
        self.no_trees = no_trees
        df = df.copy()

        # initializing our random forest as a list of all our generated trees:
        self.model = list()
        
        y = df['label'].values
        df.drop(columns=['label'], inplace=True)

        
        #df, self.one_hot = create_one_hot(df)
        #df, self.column_filter = create_column_filter(df)
        #df, self.imputation = create_imputation(df)
        
        x = df.values

        # total number of features:
        F_size = len(df.columns)


        # just to use later!
        df_with_classes = df.copy()
        df_with_classes['label'] = pd.Series(y, index=df_with_classes.index)

        def select_with_replacement():
            '''
            This function will return a newly selected with replacement sample.
            '''
            sample = pd.DataFrame(columns = df_with_classes.columns)
            selections = np.random.choice(df_with_classes.shape[0], df_with_classes.shape[0], replace=True)
            
            for selection in selections:
                sample = sample.append(df_with_classes.iloc[selection])
                

            return sample 



        for trees_i in range(0, no_trees):
            
            sample_df = select_with_replacement()
            
            y = sample_df['label'].values
            sample_df.drop(columns=['label'], inplace=True)
            x = sample_df.values

            self.model.append(DecisionTreeClassifier(max_features='log2'))
            self.model[-1].fit(x, y)

            print('tree # {} is built,'.format(trees_i))


            # I used these lines to get insight into how are my decision trees doing (commented)
            '''
            tree_desc = tree.export_graphviz(self.model[-1], out_file='1_forest/{}.dot'.format(trees_i), feature_names=list(sample_df.columns))
            dot_data = tree.export_graphviz(self.model[-1], feature_names=list(sample_df.columns), class_names=[str(c) for c in self.model[-1].classes_], filled=True, rounded=True)
            graph = graphviz.Source(dot_data)
            graph.render('1_forest/{}.gv'.format(trees_i), view=False)  
            '''
        
        
        
    
        


    
    def predict(self, df):
        test_df = df.copy()

        
        

        #test_df = apply_one_hot(test_df, self.one_hot)
        #test_df = apply_column_filter(test_df, self.column_filter)
        #test_df = apply_imputation(test_df, self.imputation)

        test_x = test_df.values

        # let's add all predictions from all trees to a single list so that we can use it later!
        # each index in this list denotes the prediction of a tree
        tree_predictions_list = list()

        for dt in self.model:
            tree_predictions = dt.predict_proba(test_x)
            tree_predictions_list.append(tree_predictions)

        self.tree_predictions_list = tree_predictions_list



        predictions = pd.DataFrame(columns=['safe' ,'vul'])

        # averaging for each instance:
        for instance_index in range(0, len(test_x)):
            
            total_negative = 0
            total_positive = 0
            
            for model_index in range(0, self.no_trees):
                total_negative += tree_predictions_list[model_index][instance_index][0]
                total_positive += tree_predictions_list[model_index][instance_index][1]
            
            

            predictions = predictions.append({
                'safe': Decimal(total_negative) / Decimal(self.no_trees),
                'vul': Decimal(total_positive) / Decimal(self.no_trees)
            }, ignore_index=True)

        return predictions






def accuracy(original_df, correctlabels):
    
    df = original_df.copy()
    
    '''
    Assumption: I assume that the accuracy is ratio `number of data instances 
    correctly classified / total no of data instances`
    Furthermore, for an instance that does not have a label with highest probability,
    I will choose the first label in column orders as the predicted label for that instance.
    '''
    
    cor_pred = list() # correct predictions list
    inc_pred = list() # incorrect predictions list
    col_list = df.columns
    for index, row in df.iterrows():
        max_prob = row.max()
        for col in col_list:
            if row[col] == max_prob:
                predicted = col
                break
        correct = correctlabels[index]
        if correct == predicted:
            cor_pred.append(index)
        else:
            inc_pred.append(index)
    
    return len(cor_pred)/ len(correctlabels)


    

In [91]:
# Random Forest 

# trying to scale the data
scaler = StandardScaler()

rf = RandomForest()

kf = KFold(n_splits=5, shuffle=True)
labels = list(df['label'])
accuracies = list()

for train_indices, test_indices in kf.split(df):
    print("TRAIN:", train_indices, "\nTEST:", test_indices)
    
    train_df = df.loc[train_indices, :].copy()
    test_df = df.loc[test_indices, :].copy()
    
    train_df.drop(columns=['tx_hash'], inplace=True)
    train_df.drop(columns=['Unnamed: 0'], inplace=True)
    
    test_df.drop(columns=['tx_hash'], inplace=True)
    test_df.drop(columns=['Unnamed: 0'], inplace=True)

    test_labels = list(test_df['label'])
    test_df.drop(columns=['label'], inplace=True)

    train_labels = list(train_df['label'])
    train_df.drop(columns=['label'], inplace=True)

    # scaling:
    scaler.fit(train_df)
    train_ndarray = scaler.transform(train_df)
    test_ndarray = scaler.transform(test_df)

    # let's add the labels to the train_df after scalling, the fit function in our Random Forest classifier needs them:
    train_df = pd.DataFrame(train_ndarray)
    train_df['label'] = train_labels
    test_df = pd.DataFrame(test_ndarray)
    
    rf.fit(train_df, no_trees=100)
    predictions = rf.predict(test_df)

    y_pred = get_ys_from_prediction(predictions)
    print(classification_report(test_labels, y_pred))

    #print(test_labels)
    #print(y_pred)

    ac = metrics.accuracy_score(test_labels, y_pred)
    #ac = accuracy(predictions, labels)
    accuracies.append(ac)

    
    print("\nAccuracy of prediction is: {}".format(ac))
    print("========================================================")
    print("========================================================")



print("*********************************************************")
print("\nAverage accuracy: {}".format(sum(accuracies) / len(accuracies)))


TRAIN: [  1   2   3   4   5   6   7   9  10  11  12  13  15  17  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  41  42
  44  45  46  48  49  51  52  53  54  56  58  59  60  61  63  64  65  66
  67  69  71  72  73  74  75  76  77  78  79  80  81  83  84  85  86  88
  90  91  92  93  95  96  97  98  99 100 102 103] 
TEST: [  0   8  14  16  18  39  40  43  47  50  55  57  62  68  70  82  87  89
  94 101 104]
tree # 0 is built,
tree # 1 is built,
tree # 2 is built,
tree # 3 is built,
tree # 4 is built,
tree # 5 is built,
tree # 6 is built,
tree # 7 is built,
tree # 8 is built,
tree # 9 is built,
tree # 10 is built,
tree # 11 is built,
tree # 12 is built,
tree # 13 is built,
tree # 14 is built,
tree # 15 is built,
tree # 16 is built,
tree # 17 is built,
tree # 18 is built,
tree # 19 is built,
tree # 20 is built,
tree # 21 is built,
tree # 22 is built,
tree # 23 is built,
tree # 24 is built,
tree # 25 is built,
tree # 26 is built,
tree # 27 is built,
tree # 

In [93]:
# Logistic Regression

# trying to scale the data
scaler = StandardScaler()

kf = KFold(n_splits=5, shuffle=True)
labels = list(df['label'])
accuracies = list()

for train_indices, test_indices in kf.split(df):
    print("TRAIN:", train_indices, "\nTEST:", test_indices)
    
    train_df = df.loc[train_indices, :].copy()
    test_df = df.loc[test_indices, :].copy()
    
    train_df.drop(columns=['tx_hash'], inplace=True)
    train_df.drop(columns=['Unnamed: 0'], inplace=True)
    
    test_df.drop(columns=['tx_hash'], inplace=True)
    test_df.drop(columns=['Unnamed: 0'], inplace=True)

    train_labels = list(train_df['label'])
    train_df.drop(columns=['label'], inplace=True)

    test_labels = list(test_df['label'])
    test_df.drop(columns=['label'], inplace=True)

    # scaling:
    scaler.fit(train_df)
    train_df = scaler.transform(train_df)
    test_df = scaler.transform(test_df)

    # training
    logreg = LogisticRegression() 
    logreg.fit(train_df, train_labels)

    # prediction
    y_pred=logreg.predict(test_df)
    #cnf_matrix = metrics.confusion_matrix(test_labels, y_pred)
    
    # measuring performance
    print(classification_report(test_labels, y_pred))  
    ac = metrics.accuracy_score(test_labels, y_pred)
    accuracies.append(ac)
    
    print("\nAccuracy of prediction is: {}".format(ac))
    print("========================================================")
    print("========================================================")



print("*********************************************************")
print("\nAverage accuracy: {}".format(sum(accuracies) / len(accuracies)))


TRAIN: [  0   1   2   3   4   6  10  11  12  14  15  16  17  18  19  20  21  23
  24  25  26  27  29  30  31  32  33  34  35  36  38  39  40  41  42  43
  44  45  46  48  50  51  52  53  54  55  56  58  59  62  63  64  65  66
  67  68  70  71  73  74  75  78  79  80  81  82  83  84  85  86  87  88
  89  91  93  95  96  97  98  99 100 101 102 104] 
TEST: [  5   7   8   9  13  22  28  37  47  49  57  60  61  69  72  76  77  90
  92  94 103]
              precision    recall  f1-score   support

        safe       0.79      1.00      0.88        11
         vul       1.00      0.70      0.82        10

    accuracy                           0.86        21
   macro avg       0.89      0.85      0.85        21
weighted avg       0.89      0.86      0.85        21


Accuracy of prediction is: 0.8571428571428571
TRAIN: [  1   2   3   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  21  22  23  24  25  26  27  28  29  32  33  35  36  37  38  39  41  43
  44  45  47  49  50  51  52  

In [94]:
# K-Nearest Neighbors

# trying to scale the data
scaler = StandardScaler()

kf = KFold(n_splits=5, shuffle=True)
labels = list(df['label'])
accuracies = list()


for train_indices, test_indices in kf.split(df):
    print("TRAIN:", train_indices, "\nTEST:", test_indices)
    
    train_df = df.loc[train_indices, :].copy()
    test_df = df.loc[test_indices, :].copy()
    
    train_df.drop(columns=['tx_hash'], inplace=True)
    train_df.drop(columns=['Unnamed: 0'], inplace=True)
    
    test_df.drop(columns=['tx_hash'], inplace=True)
    test_df.drop(columns=['Unnamed: 0'], inplace=True)

    train_labels = list(train_df['label'])
    train_df.drop(columns=['label'], inplace=True)

    test_labels = list(test_df['label'])
    test_df.drop(columns=['label'], inplace=True)

    # scaling:
    scaler.fit(train_df)
    train_df = scaler.transform(train_df)
    test_df = scaler.transform(test_df)

    classifier = KNeighborsClassifier(n_neighbors=5)

    # training
    classifier.fit(train_df, train_labels)

    # prediction
    y_pred = classifier.predict(test_df)
    #print(confusion_matrix(test_labels, y_pred))
    print(classification_report(test_labels, y_pred))    

    # measuring performance
    ac = metrics.accuracy_score(test_labels, y_pred)
    accuracies.append(ac)
    
    print("\nAccuracy of prediction is: {}".format(ac))
    print("========================================================")
    print("========================================================")



print("*********************************************************")
print("\nAverage accuracy: {}".format(sum(accuracies) / len(accuracies)))


TRAIN: [  1   5   8   9  10  11  12  13  15  16  17  19  21  22  23  24  25  26
  27  28  29  30  31  32  33  35  36  37  38  40  41  42  43  44  45  46
  47  48  50  54  55  57  58  59  61  62  63  64  66  67  69  70  72  73
  74  75  76  77  78  79  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104] 
TEST: [ 0  2  3  4  6  7 14 18 20 34 39 49 51 52 53 56 60 65 68 71 80]
              precision    recall  f1-score   support

        safe       0.65      1.00      0.79        11
         vul       1.00      0.40      0.57        10

    accuracy                           0.71        21
   macro avg       0.82      0.70      0.68        21
weighted avg       0.82      0.71      0.68        21


Accuracy of prediction is: 0.7142857142857143
TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  12  13  14  15  16  17  18
  19  20  21  22  23  25  26  28  29  30  31  33  34  35  36  37  38  39
  41  45  46  48  49  50  51  52  53  54  55  56  57

In [95]:
# Naive Bayes Classifier

# trying to scale the data
scaler = StandardScaler()

kf = KFold(n_splits=5, shuffle=True)
labels = list(df['label'])
accuracies = list()


for train_indices, test_indices in kf.split(df):
    print("TRAIN:", train_indices, "\nTEST:", test_indices)
    
    train_df = df.loc[train_indices, :].copy()
    test_df = df.loc[test_indices, :].copy()
    
    train_df.drop(columns=['tx_hash'], inplace=True)
    train_df.drop(columns=['Unnamed: 0'], inplace=True)
    
    test_df.drop(columns=['tx_hash'], inplace=True)
    test_df.drop(columns=['Unnamed: 0'], inplace=True)

    train_labels = list(train_df['label'])
    train_df.drop(columns=['label'], inplace=True)

    test_labels = list(test_df['label'])
    test_df.drop(columns=['label'], inplace=True)

    # scaling:
    scaler.fit(train_df)
    #train_df = scaler.transform(train_df)
    #test_df = scaler.transform(test_df)

    # Gaussian Naive Bayes classier:
    gnb = GaussianNB()

    # training
    gnb.fit(train_df, train_labels)

    # prediction
    y_pred = gnb.predict(test_df)
    print("********")
    print(y_pred)
    print("********")
    print(classification_report(test_labels, y_pred))    

    # measuring performance
    ac = metrics.accuracy_score(test_labels, y_pred)
    accuracies.append(ac)
    
    print("\nAccuracy of prediction is: {}".format(ac))
    print("========================================================")
    print("========================================================")



print("*********************************************************")
print("\nAverage accuracy: {}".format(sum(accuracies) / len(accuracies)))


TRAIN: [  0   1   2   4   5   6   7   8   9  12  13  14  15  16  17  18  19  20
  21  22  23  24  25  26  27  28  30  31  32  33  34  35  36  37  38  40
  42  43  46  47  48  49  50  53  54  57  58  59  60  61  62  63  64  65
  67  69  70  71  72  73  74  75  76  77  79  81  83  84  85  86  87  88
  89  90  92  93  94  95  96  97  99 100 101 102] 
TEST: [  3  10  11  29  39  41  44  45  51  52  55  56  66  68  78  80  82  91
  98 103 104]
********
['safe' 'safe' 'safe' 'safe' 'safe' 'vul' 'safe' 'safe' 'safe' 'safe'
 'vul' 'vul' 'vul' 'vul' 'safe' 'safe' 'vul' 'vul' 'vul' 'vul' 'vul']
********
              precision    recall  f1-score   support

        safe       0.91      1.00      0.95        10
         vul       1.00      0.91      0.95        11

    accuracy                           0.95        21
   macro avg       0.95      0.95      0.95        21
weighted avg       0.96      0.95      0.95        21


Accuracy of prediction is: 0.9523809523809523
TRAIN: [  0   1   2   3  

In [96]:
# SVM Classifier

# trying to scale the data
scaler = StandardScaler()

kf = KFold(n_splits=5, shuffle=True)
labels = list(df['label'])
accuracies = list()


for train_indices, test_indices in kf.split(df):
    print("TRAIN:", train_indices, "\nTEST:", test_indices)
    
    train_df = df.loc[train_indices, :].copy()
    test_df = df.loc[test_indices, :].copy()
    
    train_df.drop(columns=['tx_hash'], inplace=True)
    train_df.drop(columns=['Unnamed: 0'], inplace=True)
    
    test_df.drop(columns=['tx_hash'], inplace=True)
    test_df.drop(columns=['Unnamed: 0'], inplace=True)

    train_labels = list(train_df['label'])
    train_df.drop(columns=['label'], inplace=True)

    test_labels = list(test_df['label'])
    test_df.drop(columns=['label'], inplace=True)

    # scaling:
    scaler.fit(train_df)
    train_df = scaler.transform(train_df)
    test_df = scaler.transform(test_df)

    # SVM classier:
    clf = svm.SVC(kernel='linear')

    # training
    clf.fit(train_df, train_labels)

    # prediction
    y_pred = clf.predict(test_df)
    print(classification_report(test_labels, y_pred))    

    # measuring performance
    ac = metrics.accuracy_score(test_labels, y_pred)
    accuracies.append(ac)
    
    print("\nAccuracy of prediction is: {}".format(ac))
    print("========================================================")
    print("========================================================")



print("*********************************************************")
print("\nAverage accuracy: {}".format(sum(accuracies) / len(accuracies)))


TRAIN: [  0   1   2   4   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  21  22  23  24  25  26  27  28  29  30  33  34  36  37  38  39  40  41
  42  44  46  47  49  50  52  54  55  56  58  59  60  61  62  63  64  66
  68  69  70  72  73  75  76  77  78  80  81  82  84  86  87  88  89  90
  91  92  93  94  95  96  97 100 101 102 103 104] 
TEST: [ 3  5 20 31 32 35 43 45 48 51 53 57 65 67 71 74 79 83 85 98 99]
              precision    recall  f1-score   support

        safe       0.77      0.91      0.83        11
         vul       0.88      0.70      0.78        10

    accuracy                           0.81        21
   macro avg       0.82      0.80      0.81        21
weighted avg       0.82      0.81      0.81        21


Accuracy of prediction is: 0.8095238095238095
TRAIN: [  0   1   2   3   4   5   6   8   9  10  13  14  15  16  17  18  20  21
  23  26  27  28  29  31  32  33  34  35  36  37  38  39  40  42  43  44
  45  46  47  48  49  51  53  54  55  56  57  58  59